In [1]:
import ee
import geemap

In [ ]:
geemap.update_package()

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4, add_google_map=False)

In [3]:
states = ee.FeatureCollection("TIGER/2018/States")

In [4]:
style = {'color': 'black', 'fillColor': "00000000"}

In [5]:
Map.addLayer(states.style(**style), {}, "US States")

In [6]:
Map.add_labels(
    states,
    "STUSPS",
    font_size="12pt",
    font_color="blue",
    font_family="arial",
    font_weight="bold",
)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…